In [1]:
import sys
sys.path.append('/workspace/Documents')
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import nibabel as nb

import PSF_generation.python.functions as ff

In [2]:
main_path = '/mnt/BPM_NAS/cchen/'
ground_truth_path = os.path.join(main_path,'Dataset993_blur/labelsTs')
predict_path = os.path.join(main_path, 'project_results/nnUNet_results/Dataset993_blur/nnUNetTrainer__nnUNetPlans__2d/fold_0/inference_Dataset993_imagesTs')

In [3]:
blur_cases = ff.find_all_target_files(['*.nii.gz'], ground_truth_path)
print('Total number of cases: ', len(blur_cases))

Total number of cases:  352


In [7]:
result = []
for i in range(0,blur_cases.shape[0]):
    blur_case = blur_cases[i]
    gt = nb.load(blur_case).get_fdata(); gt = np.round(gt)
    pred = nb.load(os.path.join(predict_path, os.path.basename(blur_case))).get_fdata(); pred = np.round(pred)
   
    # dice = ff.np_categorical_dice(pred,gt,target_class = 1)

    gt = [1 if np.sum(gt) == gt.shape[0]*gt.shape[1] else 0][0]
    pred = [1 if np.sum(pred) == pred.shape[0]*pred.shape[1] else 0][0]

    if gt == pred:
        classification = 1
    else:
        classification = 0

    result.append([os.path.basename(blur_case), gt, pred, classification])
    
result = pd.DataFrame(result, columns = ['Case', 'GT', 'Pred', 'Result'])
result.to_excel(os.path.join(main_path, 'project_results/nnUNet_results/Dataset993_blur/nnUNetTrainer__nnUNetPlans__2d/fold_0/classification.xlsx'), index = False)
# result = pd.DataFrame(result, columns = ['Case', 'Dice'])
# result.to_excel(os.path.join(main_path, 'project_results/nnUNet_results/Dataset993_blur/nnUNetTrainer__nnUNetPlans__2d/fold_0/dice.xlsx'), index = False)

In [8]:
predict_blur_cases = ff.find_all_target_files(['*.nii.gz'], predict_path)
real_blur_cases = []
for i in range(0, predict_blur_cases.shape[0]):
    c = predict_blur_cases[i]

    basename = os.path.basename(c)

    not_in = True
    for j in range(0, blur_cases.shape[0]):
        if basename not in blur_cases[j]:
            real_blur_cases.append(blur_cases[j])
print('Total number of cases: ', len(real_blur_cases))

Total number of cases:  125312
